# Agriculture Crop Yield: Soil Type Prediction

By: Jaeyoung Oh

GitHub: https://github.com/BlueJayVRStudio/CSCA5632_final



##### dataset: https://www.kaggle.com/datasets/samuelotiattakorah/agriculture-crop-yield

The author of the dataset also posted a preliminary EDA. We will use this as a reference and adjust it to the needs of this project

##### EDA reference: https://www.kaggle.com/code/samuelotiattakorah/eda-on-agriculture-crop-yield

## Initial Project Setup

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# please, no warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=ConvergenceWarning)
os.environ['PYTHONWARNINGS']='ignore'

## Iterative Data Cleaning and EDA

In [3]:
# import data
df_crop_yield = pd.read_csv('./data/crop_yield.csv')

### Data Description

In [4]:
df_crop_yield.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Region                  1000000 non-null  object 
 1   Soil_Type               1000000 non-null  object 
 2   Crop                    1000000 non-null  object 
 3   Rainfall_mm             1000000 non-null  float64
 4   Temperature_Celsius     1000000 non-null  float64
 5   Fertilizer_Used         1000000 non-null  bool   
 6   Irrigation_Used         1000000 non-null  bool   
 7   Weather_Condition       1000000 non-null  object 
 8   Days_to_Harvest         1000000 non-null  int64  
 9   Yield_tons_per_hectare  1000000 non-null  float64
dtypes: bool(2), float64(3), int64(1), object(4)
memory usage: 62.9+ MB


There are no null values, so conveniently we do not have to fill in any missing values. However, we still have numerous preprocessing steps to ensure that our data is ready for clustering tasks.
> - Random sample portion of the dataset, since we will run into memory issues with 1 million data points,
> - Separate "Soil_Type" from rest of the data frame since we'll be using that as target labels,
> - "Region" and "Crop" are non-ordinal categorical data, thus we need to convert them to binary, one-hot encoding,
> - "Weather_Condition" is also categorical, but one could argue that it is ordinal since cloud cover is inversely proportional to sunlight and directly proportional to precipitation. Using just one of those two relationships should suffice,
> - For the rest of the numerical columns, we will perform quick outlier analysis and drop them as we have sufficient data. We will also ensure that no column has over-influence by using standard scaling,
> - Lastly we will check for class imbalance on the soil types to ensure that we are working with balanced dataset.

In [39]:
df_crop_yield.head(10)

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,West,Sandy,Cotton,897.077239,27.676966,False,True,Cloudy,122,6.555816
1,South,Clay,Rice,992.673282,18.026142,True,True,Rainy,140,8.527341
2,North,Loam,Barley,147.998025,29.794042,False,False,Sunny,106,1.127443
3,North,Sandy,Soybean,986.866331,16.644190,False,True,Rainy,146,6.517573
4,South,Silt,Wheat,730.379174,31.620687,True,True,Cloudy,110,7.248251
5,South,Silt,Soybean,797.471182,37.704974,False,True,Rainy,74,5.898416
6,West,Clay,Wheat,357.902357,31.593431,False,False,Rainy,90,2.652392
7,South,Sandy,Rice,441.131154,30.887107,True,True,Sunny,61,5.829542
8,North,Silt,Wheat,181.587861,26.752729,True,False,Sunny,127,2.943716
9,West,Sandy,Wheat,395.048968,17.646199,False,True,Rainy,140,3.707293


In [5]:
df_crop_yield.Soil_Type.unique()

array(['Sandy', 'Clay', 'Loam', 'Silt', 'Peaty', 'Chalky'], dtype=object)

In [40]:
df_crop_yield.Crop.unique()

array(['Cotton', 'Rice', 'Barley', 'Soybean', 'Wheat', 'Maize'],
      dtype=object)

In [18]:
sample_crop_yield = df_crop_yield.sample(10000, random_state=0)

In [45]:
model=AgglomerativeClustering(n_clusters=6, metric='euclidean', linkage='ward')
model.fit(sample_crop_yield.iloc[:, 8:])
print(model.labels_[0:20])

[0 3 5 3 0 0 1 1 3 3 5 2 3 0 1 5 1 2 2 2]


In [48]:
print(sample_crop_yield.Soil_Type[0:20])

157105      Clay
374554      Loam
688694     Peaty
265381    Chalky
955415      Silt
4280       Sandy
514674     Peaty
800352      Silt
94727      Peaty
753889      Clay
334658    Chalky
676584      Loam
796710      Silt
385008      Silt
568106      Silt
25532       Silt
840001     Peaty
726752     Sandy
939105      Silt
608171      Silt
Name: Soil_Type, dtype: object
